In [1]:
import schwabdev
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
app_key = os.getenv('SCHWAB_APP_KEY')
app_secret = os.getenv('SCHWAB_APP_SECRET')

# Create a client
client = schwabdev.Client(app_key, app_secret)

The refresh token has expired!


[Schwabdev] Open to authenticate: https://api.schwabapi.com/v1/oauth/authorize?client_id=TY6Bwjp09QzikPMKVtGzAUZwldGmqgfumA3Nz85gi6IG3gV4&redirect_uri=https://127.0.0.1


In [2]:
client.account_linked().json()

[{'accountNumber': '12674795',
  'hashValue': '121277231CBA74F809908F2AECB23B7D68C6D201905F61C5FBEAD716D2C0F250'}]

In [3]:
# create a variable called accounthash based on the hash value from the account_linked endpoint
accounthash = client.account_linked().json()[0].get('hashValue')

In [4]:
accounthash

'121277231CBA74F809908F2AECB23B7D68C6D201905F61C5FBEAD716D2C0F250'

In [5]:
import pandas as pd
data=client.account_details_all().json()

In [6]:
data

[{'securitiesAccount': {'type': 'MARGIN',
   'accountNumber': '12674795',
   'roundTrips': 0,
   'isDayTrader': False,
   'isClosingOnlyRestricted': False,
   'pfcbFlag': False,
   'initialBalances': {'accruedInterest': 0.0,
    'availableFundsNonMarginableTrade': 9402.01,
    'bondValue': 37608.04,
    'buyingPower': 31340.03,
    'cashBalance': 9402.01,
    'cashAvailableForTrading': 0.0,
    'cashReceipts': 0.0,
    'dayTradingBuyingPower': 115966.0,
    'dayTradingBuyingPowerCall': 0.0,
    'dayTradingEquityCall': 0.0,
    'equity': 40007.26,
    'equityPercentage': 100.0,
    'liquidationValue': 40007.26,
    'longMarginValue': 26119.67,
    'longOptionMarketValue': 4485.58,
    'longStockValue': 26119.67,
    'maintenanceCall': 0.0,
    'maintenanceRequirement': 26120.0,
    'margin': 0.0,
    'marginEquity': 26119.67,
    'moneyMarketFund': 0.0,
    'mutualFundValue': 9402.01,
    'regTCall': 0.0,
    'shortMarginValue': 0.0,
    'shortOptionMarketValue': 0.0,
    'shortStockVal

In [7]:
df_accounts=pd.json_normalize(data)

In [8]:
df_accounts

,securitiesAccount.type,securitiesAccount.accountNumber,securitiesAccount.roundTrips,securitiesAccount.isDayTrader,securitiesAccount.isClosingOnlyRestricted,securitiesAccount.pfcbFlag,securitiesAccount.initialBalances.accruedInterest,securitiesAccount.initialBalances.availableFundsNonMarginableTrade,securitiesAccount.initialBalances.bondValue,securitiesAccount.initialBalances.buyingPower,...,securitiesAccount.projectedBalances.availableFundsNonMarginableTrade,securitiesAccount.projectedBalances.buyingPower,securitiesAccount.projectedBalances.dayTradingBuyingPower,securitiesAccount.projectedBalances.dayTradingBuyingPowerCall,securitiesAccount.projectedBalances.maintenanceCall,securitiesAccount.projectedBalances.regTCall,securitiesAccount.projectedBalances.isInCall,securitiesAccount.projectedBalances.stockBuyingPower,aggregatedBalance.currentLiquidationValue,aggregatedBalance.liquidationValue
0,MARGIN,12674795,0,False,False,False,0.0,9402.01,37608.04,31340.03,...,9402.01,9402.01,115966.0,0.0,0.0,0.0,False,9402.01,40209.18,40209.18


In [9]:
import datetime 

In [10]:
trade=client.transactions(accounthash, datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(days=100), datetime.datetime.now(datetime.timezone.utc), types="TRADE").json()

In [11]:
trade

[{'activityId': 104228197590,
  'time': '2025-10-06T22:08:12+0000',
  'description': 'SPDR BLOOMBERG 1 3 MNTH T BLL ETF',
  'accountNumber': '12674795',
  'type': 'TRADE',
  'status': 'VALID',
  'subAccount': 'MARGIN',
  'tradeDate': '2025-10-06T04:00:00+0000',
  'settlementDate': '2025-10-06T04:00:00+0000',
  'positionId': 2808435746,
  'netAmount': -71.42,
  'transferItems': [{'instrument': {'assetType': 'COLLECTIVE_INVESTMENT',
     'status': 'ACTIVE',
     'symbol': 'BIL',
     'description': 'SPDR Blmbg 1-3 Mth T-Bill ETF',
     'instrumentId': 65013305,
     'closingPrice': 91.72,
     'type': 'EXCHANGE_TRADED_FUND'},
    'amount': 0.7805,
    'cost': -71.42,
    'price': 91.51,
    'positionEffect': 'OPENING'}]},
 {'activityId': 107578756622,
  'time': '2025-11-26T14:30:04+0000',
  'accountNumber': '12674795',
  'type': 'TRADE',
  'status': 'VALID',
  'subAccount': 'MARGIN',
  'tradeDate': '2025-11-26T14:30:04+0000',
  'positionId': 3087195794,
  'orderId': 1004779596747,
  'net

In [12]:
trade1=pd.json_normalize(trade)

In [13]:
df2=trade1.explode('transferItems').reset_index(drop=True)

In [14]:
pd.concat([df2.drop(columns=['transferItems']), pd.json_normalize(df2['transferItems'])], axis=1)

,activityId,time,description,accountNumber,type,status,subAccount,tradeDate,settlementDate,positionId,...,instrument.closingPrice,instrument.type,feeType,instrument.expirationDate,instrument.optionDeliverables,instrument.optionPremiumMultiplier,instrument.putCall,instrument.strikePrice,instrument.underlyingSymbol,instrument.underlyingCusip
0,104228197590,2025-10-06T22:08:12+0000,SPDR BLOOMBERG 1 3 MNTH T BLL ETF,12674795,TRADE,VALID,MARGIN,2025-10-06T04:00:00+0000,2025-10-06T04:00:00+0000,2.808436e+09,...,91.7200,EXCHANGE_TRADED_FUND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,107578756622,2025-11-26T14:30:04+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-11-26T14:30:04+0000,NaN,3.087196e+09,...,0.0000,NaN,COMMISSION,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,107578756622,2025-11-26T14:30:04+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-11-26T14:30:04+0000,NaN,3.087196e+09,...,0.0000,NaN,SEC_FEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,107578756622,2025-11-26T14:30:04+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-11-26T14:30:04+0000,NaN,3.087196e+09,...,0.0000,NaN,OPT_REG_FEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,107578756622,2025-11-26T14:30:04+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-11-26T14:30:04+0000,NaN,3.087196e+09,...,0.0000,NaN,TAF_FEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,107578756622,2025-11-26T14:30:04+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-11-26T14:30:04+0000,NaN,3.087196e+09,...,3.2000,VANILLA,NaN,2026-06-18T04:00:00+0000,"[{'rootSymbol': 'XLV', 'strikePercent': 100, '...",100.0,CALL,170.0,XLV,NaN
6,104611865028,2025-10-10T19:57:17+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-10-10T19:57:17+0000,NaN,2.776388e+09,...,0.0000,NaN,COMMISSION,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,104611865028,2025-10-10T19:57:17+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-10-10T19:57:17+0000,NaN,2.776388e+09,...,0.0000,NaN,SEC_FEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,104611865028,2025-10-10T19:57:17+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-10-10T19:57:17+0000,NaN,2.776388e+09,...,0.0000,NaN,OPT_REG_FEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,104611865028,2025-10-10T19:57:17+0000,NaN,12674795,TRADE,VALID,MARGIN,2025-10-10T19:57:17+0000,NaN,2.776388e+09,...,0.0000,NaN,TAF_FEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
pd.json_normalize(client.option_expiration_chain("AAPL").json()['expirationList'])


,expirationDate,daysToExpiration,expirationType,settlementType,optionRoots,standard
0,2025-12-05,5,W,P,AAPL,True
1,2025-12-12,12,W,P,AAPL,True
2,2025-12-19,19,S,P,AAPL,True
3,2025-12-26,26,W,P,AAPL,True
4,2026-01-02,33,W,P,AAPL,True
5,2026-01-09,40,W,P,AAPL,True
6,2026-01-16,47,S,P,AAPL,True
7,2026-02-20,82,S,P,AAPL,True
8,2026-03-20,110,S,P,AAPL,True
9,2026-04-17,138,S,P,AAPL,True


In [16]:
client.option_chains("AAPL", "Call",strikeCount=10).json()

{'errors': None}

In [17]:
AAPL_chain=client.option_chains("AAPL", contractType="CALL", strikeCount=None, includeUnderlyingQuote=None, strategy=None,
               interval=None, strike=None, range=None, fromDate=None, toDate=None, volatility=None, underlyingPrice=None,
               interestRate=None, daysToExpiration=None, expMonth="DEC", optionType=None, entitlement=None).json()

In [18]:
AAPL_chain

{'symbol': 'AAPL',
 'status': 'SUCCESS',
 'underlying': None,
 'strategy': 'SINGLE',
 'interval': 0.0,
 'isDelayed': False,
 'isIndex': False,
 'interestRate': 3.697,
 'underlyingPrice': 278.85,
 'volatility': 29.0,
 'daysToExpiration': 5.0,
 'dividendYield': 0.373,
 'numberOfContracts': 467,
 'assetMainType': 'EQUITY',
 'assetSubType': 'COE',
 'isChainTruncated': False,
 'callExpDateMap': {'2025-12-05:5': {'110.0': [{'putCall': 'CALL',
     'symbol': 'AAPL  251205C00110000',
     'description': 'AAPL 12/05/2025 110.00 C',
     'exchangeName': 'OPR',
     'bid': 167.25,
     'ask': 169.5,
     'last': 168.32,
     'mark': 168.38,
     'bidSize': 141,
     'askSize': 143,
     'bidAskSize': '141X143',
     'lastSize': 3,
     'highPrice': 0.0,
     'lowPrice': 0.0,
     'openPrice': 0.0,
     'closePrice': 168.96,
     'totalVolume': 0,
     'tradeTimeInLong': 1764183176753,
     'quoteTimeInLong': 1764352799976,
     'netChange': -0.64,
     'volatility': 203.628,
     'delta': 1.0,
  

In [19]:
option_chains_alt=client.option_chains("AAPL", contractType="CALL", strikeCount=None, includeUnderlyingQuote=None,   strategy=None,interval=None, strike=None, range=None, fromDate=None, toDate=None, volatility=None, underlyingPrice=None,
interestRate=None, daysToExpiration=None, expMonth="DEC", optionType=None, entitlement=None)

In [20]:
option_chains_alt.json()

{'symbol': 'AAPL',
 'status': 'SUCCESS',
 'underlying': None,
 'strategy': 'SINGLE',
 'interval': 0.0,
 'isDelayed': False,
 'isIndex': False,
 'interestRate': 3.697,
 'underlyingPrice': 278.85,
 'volatility': 29.0,
 'daysToExpiration': 5.0,
 'dividendYield': 0.373,
 'numberOfContracts': 467,
 'assetMainType': 'EQUITY',
 'assetSubType': 'COE',
 'isChainTruncated': False,
 'callExpDateMap': {'2025-12-05:5': {'110.0': [{'putCall': 'CALL',
     'symbol': 'AAPL  251205C00110000',
     'description': 'AAPL 12/05/2025 110.00 C',
     'exchangeName': 'OPR',
     'bid': 167.25,
     'ask': 169.5,
     'last': 168.32,
     'mark': 168.38,
     'bidSize': 141,
     'askSize': 143,
     'bidAskSize': '141X143',
     'lastSize': 3,
     'highPrice': 0.0,
     'lowPrice': 0.0,
     'openPrice': 0.0,
     'closePrice': 168.96,
     'totalVolume': 0,
     'tradeTimeInLong': 1764183176753,
     'quoteTimeInLong': 1764352799976,
     'netChange': -0.64,
     'volatility': 203.628,
     'delta': 1.0,
  

In [21]:
pd.json_normalize(AAPL_chain)

,symbol,status,underlying,strategy,interval,isDelayed,isIndex,interestRate,underlyingPrice,volatility,...,callExpDateMap.2027-12-17:747.440.0,callExpDateMap.2027-12-17:747.450.0,callExpDateMap.2027-12-17:747.460.0,callExpDateMap.2027-12-17:747.470.0,callExpDateMap.2027-12-17:747.480.0,callExpDateMap.2027-12-17:747.490.0,callExpDateMap.2027-12-17:747.500.0,callExpDateMap.2027-12-17:747.510.0,callExpDateMap.2027-12-17:747.520.0,callExpDateMap.2027-12-17:747.530.0
0,AAPL,SUCCESS,None,SINGLE,0.0,False,False,3.697,278.85,29.0,...,"[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0...","[{'putCall': 'CALL', 'symbol': 'AAPL 271217C0..."


In [22]:
AAPL_chain

{'symbol': 'AAPL',
 'status': 'SUCCESS',
 'underlying': None,
 'strategy': 'SINGLE',
 'interval': 0.0,
 'isDelayed': False,
 'isIndex': False,
 'interestRate': 3.697,
 'underlyingPrice': 278.85,
 'volatility': 29.0,
 'daysToExpiration': 5.0,
 'dividendYield': 0.373,
 'numberOfContracts': 467,
 'assetMainType': 'EQUITY',
 'assetSubType': 'COE',
 'isChainTruncated': False,
 'callExpDateMap': {'2025-12-05:5': {'110.0': [{'putCall': 'CALL',
     'symbol': 'AAPL  251205C00110000',
     'description': 'AAPL 12/05/2025 110.00 C',
     'exchangeName': 'OPR',
     'bid': 167.25,
     'ask': 169.5,
     'last': 168.32,
     'mark': 168.38,
     'bidSize': 141,
     'askSize': 143,
     'bidAskSize': '141X143',
     'lastSize': 3,
     'highPrice': 0.0,
     'lowPrice': 0.0,
     'openPrice': 0.0,
     'closePrice': 168.96,
     'totalVolume': 0,
     'tradeTimeInLong': 1764183176753,
     'quoteTimeInLong': 1764352799976,
     'netChange': -0.64,
     'volatility': 203.628,
     'delta': 1.0,
  